<a href="https://colab.research.google.com/github/abisubramanya27/CS6910_Assignment2/blob/main/partA/src/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls '/content/gdrive/MyDrive/assignments/cs6910/A2/Data/nature_12K.zip (Unzipped Files)/inaturalist_12K/train/Amphibia'

In [21]:
%cd gdrive/MyDrive/assignments/cs6910/A2/Data
!pwd

/content/gdrive/.shortcut-targets-by-id/1H4LUGHYi_ivI7p5xWjyrjFojgIGEh9Jd/cs6910/A2/Data
/content/gdrive/.shortcut-targets-by-id/1H4LUGHYi_ivI7p5xWjyrjFojgIGEh9Jd/cs6910/A2/Data


In [41]:
%pip install split-folders

In [43]:
# import matplotlib.pyplot as plt
# import os

# # sample_imgs = np.array([])
# # for  
# sample_img = plt.imread('./nature_12K.zip (Unzipped Files)/inaturalist_12K/train/Amphibia/8452973d317287a2ac3082e7e9cf1ee2.jpg')
# print(sample_img.shape)
# plt.imshow(sample_img)
import splitfolders

splitfolders.ratio('./inaturalist_12K/train', output='./inaturalist_12K/output', seed=1337, ratio=(.9, .1), group_prefix=None)

Copying files: 10010 files [1:46:23,  1.57 files/s]


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


In [40]:
# Hyperparameters for building the model for Part - A
K_list_1 = [32, 32, 32, 32, 64, 64, 64, 64, 128, 128]       # List of number of filters in each non FC layer
F_list_1 = [11, 3, 5, 3, 3, 3, 3, 3, 3, 3]                  # List of size of filters in each non FC layer  
no_neurons_dense_1 = 1024                                   # Number of neurons in the dense FC layer
activation_fn_list_1 = ['relu']*6                           # List of activation function in each convolution and FC layer
P_list_1 = ['valid']*10                                     # List of padding options in each non FC layer ('valid' : no padding, 'same' : padding to make input and output same dimensions)
S_list_1 = [4, 2, 1, 1, 1, 1, 1, 2, 1, 1]                   # List of number of strides in each convolution layer
inp_img_shape_1 = (227, 227, 3)                             # Shape of input image from data
no_classes_1 = 10                                           # Number of output classes in the classification problem

def build_model_partA(inp_img_shape, K_list, F_list, no_neurons_dense, no_classes = 10, activation_fn_list = ['relu']*6, 
                      P_list = ['valid']*10, S_list = [1]*10):
    '''
    Function to build the model comprising (5 conv+relu+maxpooling layers + 1 dense FC layer) for part A in keras
    Arguments :
        inp_img_shape - shape of input image
    '''
    model = Sequential()
    # First layer
    model.add(Conv2D(filters = K_list[0], kernel_size = (F_list[0], F_list[0]), activation = activation_fn_list[0], 
                        strides = (S_list[0], S_list[0]), padding = P_list[0], input_shape = inp_img_shape))
    model.add(MaxPooling2D(pool_size=(F_list[1], F_list[1]), strides = (S_list[1], S_list[1]), padding = P_list[1]))

    # 4 Conv-relu-MaxPooling layers
    for l in range(1, 5):
        model.add(Conv2D(filters = K_list[2*l], kernel_size = (F_list[2*l], F_list[2*l]), activation = activation_fn_list[l], 
                        strides = (S_list[2*l], S_list[2*l]), padding = P_list[2*l], input_shape = inp_img_shape))
        model.add(MaxPooling2D(pool_size = (F_list[2*l+1], F_list[2*l+1]), strides = (S_list[2*l+1], S_list[2*l+1]), padding = P_list[2*l+1]))
    
    # 1 dense FC layer
    model.add(Flatten())
    model.add(Dense(units = no_neurons_dense, activation = activation_fn_list[5]))

    # Output layer
    model.add(Dense(units = no_classes, activation = 'softmax'))

    return model


# PART - A, Question 1
modelA = build_model_partA(inp_img_shape_1, K_list_1, F_list_1, no_neurons_dense_1, no_classes_1, 
                           activation_fn_list_1, P_list_1, S_list_1)
modelA.summary()


Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_52 (Conv2D)           (None, 55, 55, 32)        11648     
_________________________________________________________________
max_pooling2d_51 (MaxPooling (None, 27, 27, 32)        0         
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 23, 23, 32)        25632     
_________________________________________________________________
max_pooling2d_52 (MaxPooling (None, 21, 21, 32)        0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 19, 19, 64)        18496     
_________________________________________________________________
max_pooling2d_53 (MaxPooling (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 15, 15, 64)      